In [1]:
import pandas as pd
import csv
import os
import re

In [2]:
filter_columns = {
    'left_only': ['組別_', '檔案類型_', '檔案資料夾_', '大小_', '修改日期_'],
    'right_only': ['組別_', '檔案類型_', '檔案資料夾_', '大小_', '修改日期_'],
}

In [3]:
rename_columns = {
    'left_only': '來源路徑',
    'right_only': '比較路徑',
}

In [4]:
log_df = pd.read_csv('E:/.developing_env/duplicated_files/group_by/.backup_comp/log_different.csv')
log_df.drop(['備註', 'Unnamed: 0'], axis=1, inplace=True)
log_df.set_index('結果', drop=True, inplace=True)
log_df

,檔案資料夾,來源路徑,比較路徑,次數,來源長度,比較長度
結果,,,,,,
1,artwork,D:\MAME Games\artwork\,D:\MAME Games\mame0238b_64bit\artwork\,1,16,17
2,cfg,D:\MAME Games\cfg\,D:\MAME Games\mame0238b_64bit\cfg\,1,1,3
3,chains,D:\MAME Games\artwork\bgfx\chains\,D:\MAME Games\bgfx\chains\,1,3,2
4,chains,D:\MAME Games\artwork\bgfx\chains\,D:\MAME Games\mame0238b_64bit\bgfx\chains\,3,3,2
5,crt-geom,D:\MAME Games\bgfx\shaders\dx11\chains\crt-geom\,D:\MAME Games\mame0238b_64bit\bgfx\shaders\dx1...,6,2,3
...,...,...,...,...,...,...
21600,下載書籍,D:\_下載的文件待歸類\下載書籍\,Z:\新增書籍文件\下載書籍\,1,2,5
21601,不再聽不懂 圖解網站建置與開發,D:\_eBooks 電子書 Java+JS+HTML+CSS+RWD+Node\不再聽不懂...,Z:\新增書籍文件\資訊書籍\不再聽不懂 圖解網站建置與開發\,1,3,2
21602,讓響應式 (RWD) 網頁設計變簡單 Bootstrap 開發速成 (第二版),D:\_eBooks 電子書 Java+JS+HTML+CSS+RWD+Node\讓響應式 ...,Z:\新增書籍文件\資訊書籍\讓響應式 (RWD) 網頁設計變簡單 Bootstrap 開發...,1,4,3


In [5]:
df = pd.read_csv('df_index.csv')
df

,磁碟機根目錄,組別,檔案類型,磁碟機,根目錄,子目錄,檔案資料夾,檔案名稱,路徑,大小,新增日期,修改日期
0,Z:\8分鐘搞懂理財知識_新手教室 Youtube 投資影片教學,1,.mp4,Z:,8分鐘搞懂理財知識_新手教室 Youtube 投資影片教學,NaN,8分鐘搞懂理財知識_新手教室 Youtube 投資影片教學,8分鐘搞懂技術分析：RSI的重要價值-WUwaMVSCDYY.mp4,Z:\8分鐘搞懂理財知識_新手教室 Youtube 投資影片教學\,25612876,2021/5/10 下午 05:08:28,2018/11/14 下午 02:12:09
1,Z:\8分鐘搞懂理財知識_重新整理過,1,.mp4,Z:,8分鐘搞懂理財知識_重新整理過,NaN,8分鐘搞懂理財知識_重新整理過,2015-08-25_2152_技術分析教學_快速搞懂RSI技術指標.mp4,Z:\8分鐘搞懂理財知識_重新整理過\,25612876,2021/5/11 上午 03:06:53,2021/5/11 上午 02:58:44
2,Z:\Youtube 投資影片教學,1,.mp4,Z:,Youtube 投資影片教學,NaN,Youtube 投資影片教學,8分鐘搞懂技術分析：RSI的重要價值-WUwaMVSCDYY.mp4,Z:\Youtube 投資影片教學\,25612876,2021/5/10 下午 04:04:52,2018/11/14 下午 02:12:09
3,D:\8分鐘搞懂理財知識,1,.mp4,D:,8分鐘搞懂理財知識,_8分鐘搞懂理財知識_原來的 新手教室 Youtube 投資影片教學\,_8分鐘搞懂理財知識_原來的 新手教室 Youtube 投資影片教學,8分鐘搞懂技術分析：RSI的重要價值-WUwaMVSCDYY.mp4,D:\8分鐘搞懂理財知識\_8分鐘搞懂理財知識_原來的 新手教室 Youtube 投資影片教學\,25612876,2021/8/18 上午 02:37:16,2018/11/14 下午 02:12:09
4,D:\8分鐘搞懂理財知識,1,.mp4,D:,8分鐘搞懂理財知識,_8分鐘搞懂理財知識_整理的 Youtube 影片\,_8分鐘搞懂理財知識_整理的 Youtube 影片,2015-08-25_2152_技術分析教學_快速搞懂RSI技術指標.mp4,D:\8分鐘搞懂理財知識\_8分鐘搞懂理財知識_整理的 Youtube 影片\,25612876,2021/8/18 上午 02:38:06,2021/5/11 上午 02:58:44
...,...,...,...,...,...,...,...,...,...,...,...,...
468164,Z:\資訊書籍文件,167210,.props,Z:,資訊書籍文件,Visual C# 一次就懂 ASP.NET MVC5.X 網站開發 Web 應用的經典實務...,net46,Microsoft.CodeDom.Providers.DotNetCompilerPlat...,Z:\資訊書籍文件\Visual C# 一次就懂 ASP.NET MVC5.X 網站開發 W...,5530,2020/3/7 下午 08:06:24,2018/5/24 上午 05:38:22
468165,Z:\資訊書籍文件,167210,.props,Z:,資訊書籍文件,Visual C# 一次就懂 ASP.NET MVC5.X 網站開發 Web 應用的經典實務...,net45,Microsoft.CodeDom.Providers.DotNetCompilerPlat...,Z:\資訊書籍文件\Visual C# 一次就懂 ASP.NET MVC5.X 網站開發 W...,5530,2020/3/7 下午 08:09:52,2018/5/24 上午 05:38:22
468166,Z:\資訊書籍文件,167210,.props,Z:,資訊書籍文件,Visual C# 一次就懂 ASP.NET MVC5.X 網站開發 Web 應用的經典實務...,net46,Microsoft.CodeDom.Providers.DotNetCompilerPlat...,Z:\資訊書籍文件\Visual C# 一次就懂 ASP.NET MVC5.X 網站開發 W...,5530,2020/3/7 下午 08:09:53,2018/5/24 上午 05:38:22
468167,Z:\資訊書籍文件,167211,.strings,Z:,資訊書籍文件,iOS 13 程式設計實戰 Swift 5.1SwiftUI 框架 快速上手的開發技巧 20...,zh-Hant.lproj,LaunchScreen.strings,Z:\資訊書籍文件\iOS 13 程式設計實戰 Swift 5.1SwiftUI 框架 快速...,1,2020/2/10 上午 07:58:08,2019/7/31 上午 07:20:26


In [6]:
consolidated_df = pd.DataFrame()
for index, row in log_df.iterrows():
    compare_result = pd.read_csv(f'E:/.developing_env/duplicated_files/group_by/compare_result/{index}.csv')
    compare_result.rename(columns={'Unnamed: 0':'索引號'}, inplace=True)
    compare_result.set_index('檔案名稱', inplace=True)
    group_compare = compare_result.groupby('_merge')
    for filter_name in group_compare.groups.keys():
        get_frame = group_compare.get_group(filter_name)[filter_columns[filter_name]]
        get_frame.columns = [column.rstrip('_') for column in get_frame.columns]
        for group in list(set((get_frame['組別'].unique()))):
            isolated_frame = df[df['組別'] == group].set_index('組別').copy()
            isolated_frame['來源路徑'] = log_df.loc[index]['來源路徑']
            isolated_frame['比較路徑'] = log_df.loc[index]['比較路徑']
            isolated_frame['差異路徑'] = rename_columns[filter_name]
            isolated_frame['差異檔案'] = ','.join(get_frame.index.to_list())
            consolidated_df = pd.concat([consolidated_df, isolated_frame])

KeyboardInterrupt: 

In [8]:
consolidated_df[consolidated_df['差異路徑']!='比較路徑']

,磁碟機根目錄,檔案類型,磁碟機,根目錄,子目錄,檔案資料夾,檔案名稱,路徑,大小,新增日期,修改日期,來源路徑,比較路徑,差異路徑,差異檔案
組別,,,,,,,,,,,,,,,


In [9]:
consolidated_df

,磁碟機根目錄,檔案類型,磁碟機,根目錄,子目錄,檔案資料夾,檔案名稱,路徑,大小,新增日期,修改日期,來源路徑,比較路徑,差異路徑,差異檔案
組別,,,,,,,,,,,,,,,
2878,D:\MAME Games,.txt,D:,MAME Games,artwork\,artwork,dir.txt,D:\MAME Games\artwork\,26,2021/12/18 下午 05:23:42,2020/5/19 上午 11:11:59,D:\MAME Games\artwork\,D:\MAME Games\mame0238b_64bit\artwork\,比較路徑,dir (2).txt
2878,D:\MAME Games,.txt,D:,MAME Games,mame0238b_64bit\artwork\,artwork,dir (2).txt,D:\MAME Games\mame0238b_64bit\artwork\,26,2021/12/18 下午 05:24:21,1998/10/4 上午 02:20:58,D:\MAME Games\artwork\,D:\MAME Games\mame0238b_64bit\artwork\,比較路徑,dir (2).txt
2878,D:\MAME Games,.txt,D:,MAME Games,mame0238b_64bit\artwork\,artwork,dir.txt,D:\MAME Games\mame0238b_64bit\artwork\,26,2021/12/18 下午 05:46:34,2021/11/24 上午 03:40:53,D:\MAME Games\artwork\,D:\MAME Games\mame0238b_64bit\artwork\,比較路徑,dir (2).txt
3531,D:\MAME Games,.cfg,D:,MAME Games,mame0238b_64bit\cfg\,cfg,dkong.cfg,D:\MAME Games\mame0238b_64bit\cfg\,1416,2021/12/18 下午 05:24:21,2021/12/19 下午 05:39:17,D:\MAME Games\cfg\,D:\MAME Games\mame0238b_64bit\cfg\,比較路徑,"dkong.cfg,dkongjr.cfg"
3531,D:\MAME Games,.cfg,D:,MAME Games,mame0238b_64bit\cfg\,cfg,dkongjr.cfg,D:\MAME Games\mame0238b_64bit\cfg\,1416,2021/12/18 下午 05:24:21,2014/2/11 下午 03:35:02,D:\MAME Games\cfg\,D:\MAME Games\mame0238b_64bit\cfg\,比較路徑,"dkong.cfg,dkongjr.cfg"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6678,D:\_eBooks 電子書 AI & BigData,.mat,D:,_eBooks 電子書 AI & BigData,AI + 大數據 用 Python 玩轉金融遊戲的量化交易\AI + 大數據 用 Pytho...,data,corrupted_zlib_checksum.mat,D:\_eBooks 電子書 AI & BigData\AI + 大數據 用 Python ...,276,2021/8/18 上午 01:13:30,2016/8/26 上午 03:09:32,D:\_eBooks 電子書 AI & BigData\AI + 大數據 用 Python ...,E:\.ongoing_projects\flask_stock\financial_rep...,比較路徑,"bad_miuint32.mat,bad_miutf8_array_name.mat,big..."
6679,E:\.ongoing_projects,.mat,E:,.ongoing_projects,flask_stock\financial_report_parser\venv\Lib\s...,data,corrupted_zlib_data.mat,E:\.ongoing_projects\flask_stock\financial_rep...,3451,2021/8/18 上午 01:17:35,2021/7/3 下午 07:50:34,D:\_eBooks 電子書 AI & BigData\AI + 大數據 用 Python ...,E:\.ongoing_projects\flask_stock\financial_rep...,比較路徑,"bad_miuint32.mat,bad_miutf8_array_name.mat,big..."
6679,D:\_eBooks 電子書 AI & BigData,.mat,D:,_eBooks 電子書 AI & BigData,AI + 大數據 用 Python 玩轉金融遊戲的量化交易\AI + 大數據 用 Pytho...,data,corrupted_zlib_data.mat,D:\_eBooks 電子書 AI & BigData\AI + 大數據 用 Python ...,3451,2021/8/18 上午 01:13:30,2016/8/26 上午 03:09:32,D:\_eBooks 電子書 AI & BigData\AI + 大數據 用 Python ...,E:\.ongoing_projects\flask_stock\financial_rep...,比較路徑,"bad_miuint32.mat,bad_miutf8_array_name.mat,big..."


In [10]:
index

4191

In [11]:
consolidated_df.to_csv('consolidated_df.csv', encoding='utf-8-sig')